# Script Documentation

This document describes the functionality of a Databricks script designed to process catalog pairs, create schemas, set ownership, and clone tables or create views, with comprehensive logging of all actions. The script uses parallel processing for efficiency and logs results in a structured table.

## Parameters Overview

The notebook accepts the following parameters to drive the logic for processing catalog pairs, schema ownership, parallel execution, and logging:

1. **catalog_pair_names_list**:
   - **Type**: List of strings
   - **Description**: A list of strings in the format `source_catalog:target_catalog`, specifying pairs of source and target catalogs. The script clones tables or creates views from the source catalog to the target catalog.
   - **Example Value**: `['_eo_amperity:dr_eo_amperity', 'accuweather_daily_and_hourly_forecasts_u_s_postal_codes_sample:dr_accuweather', 'test_not:dr_test_not', 'satya_share:satya_dr_catalog']`
   - **Usage**: Iterates over catalog pairs to replicate schemas, tables, and views from the source to the target catalog.

2. **target_schema_owner_list**:
   - **Type**: List of strings
   - **Description**: A list of strings in the format `catalog.schema:owner_group`, defining ownership for schemas in the target catalog. Matching schemas are assigned the specified `owner_group`.
   - **Example Value**: `['dr_eo_amperity.bronze:grp_test_clt_usa', 'dr_eo_amperity.silver:grp_test_clt_usa', 'dr_eo_amperity.default:grp_test_clt_usa']`
   - **Usage**: Assigns ownership to schemas in the target catalog after creation to enforce access control.

3. **max_workers**:
   - **Type**: Integer
   - **Description**: Specifies the maximum number of worker threads for parallel processing of tables and views using `ThreadPoolExecutor`.
   - **Example Value**: `5`
   - **Usage**: Controls parallelism to optimize performance during table/view processing.

4. **log_table_name**:
   - **Type**: String
   - **Description**: The fully qualified name of the table where logs are stored.
   - **Example Value**: `users.satyendranath_sure.dr_log_table_name`
   - **Usage**: Stores detailed logs of all actions (e.g., catalog creation, schema ownership changes, table cloning) in a structured format.

### Example Parameter Usage

```python
catalog_pair_names_list = [
    '_eo_amperity:dr_eo_amperity',
    'accuweather_daily_and_hourly_forecasts_u_s_postal_codes_sample:dr_accuweather',
    'test_not:dr_test_not',
    'satya_share:satya_dr_catalog'
]
target_schema_owner_list = [
    'dr_eo_amperity.bronze:grp_test_clt_usa',
    'dr_eo_amperity.silver:grp_test_clt_usa',
    'dr_eo_amperity.default:grp_test_clt_usa'
]
max_workers = 5
log_table_name = 'users.satyendranath_sure.dr_log_table_name'

## Script Functions

The script is organized into several functions, each with a specific role in the overall workflow.

#### `main(catalog_pair_names_list, target_schema_owner_list, max_workers)`
This is the entry point of the script. It first sequentially collects all table and view tasks to be performed. It then uses a `ThreadPoolExecutor` to process these tasks in parallel, with the number of workers limited by `max_workers`.

#### `collect_all_table_tasks(catalog_pair_names_list, target_schema_owner_list)`
This function iterates through each `catalog_pair_names_list`. For each pair, it first creates the target catalog, then retrieves all schemas from the source catalog. It creates each schema in the target catalog and sets its ownership based on `target_schema_owner_list`. Finally, it collects a list of all tables and views from the source schemas to be processed later.

#### `create_catalog_if_not_exists(target_catalog)`
This function creates the specified catalog if it doesn't already exist. All actions are logged to the designated log table.

#### `create_schema_and_set_owner(target_catalog, schema, target_schema_owner_list)`
This function first creates the schema within the target catalog. It then attempts to set the schema's ownership to the owner group specified in `target_schema_owner_list` if a matching entry is found.

#### `process_table_or_view(source_catalog, target_catalog, schema_name, table_row)`
This function is responsible for cloning tables and creating views. It first determines whether the object is a table or a view. If it's a table, it uses a `CREATE OR REPLACE TABLE... CLONE` command. If it's a view, it uses `CREATE OR REPLACE VIEW` with the original view definition. Temporary tables and views are skipped.


## Log Table Schema

The script creates and populates a log table, which records the results of every action performed. This is useful for auditing and troubleshooting.

| Column       | Type                       | Description                                       |
|--------------|----------------------------|---------------------------------------------------|
| `entity_type`  | `STRING`                   | The type of object being processed (e.g., `catalog`, `schema`, `table`, `view`). |
| `entity_name`  | `STRING`                   | The fully qualified name of the object.           |
| `action`       | `STRING`                   | The action taken (e.g., `create`, `clone`, `change_ownership`, `skip`). |
| `status`       | `STRING`                   | The outcome of the action (`success`, `error`, `skipped`). |
| `message`      | `STRING`                   | A detailed message describing the action's result. |
| `timestamp`    | `TIMESTAMP`                | The time the action was logged.                   |
| `results_data` | `ARRAY<STRUCT<...>>`       | Additional key-value pairs of data from successful SQL commands. |


## Execute the main function
main(catalog_pair_names_list, target_schema_owner_list, max_workers)

In [0]:
# dbutils.widgets.removeAll()
dbutils.widgets.text("catalog_names", "")#source and target catalog names like dictionary
dbutils.widgets.text("target_schema_owner", "")
dbutils.widgets.text("max_workers", "5")
dbutils.widgets.text("log_table_name", "users.satyendranath_sure.dr_log_table_name")

In [0]:
catalog_pair_names_list = [catalog_pair.strip() for catalog_pair in dbutils.widgets.get("catalog_names").split(",")]
target_schema_owner_list = [catalog_schema_pair.strip() for catalog_schema_pair in dbutils.widgets.get("target_schema_owner").split(",")]
max_workers = int(dbutils.widgets.get("max_workers"))
log_table_name = dbutils.widgets.get("log_table_name")
print(catalog_pair_names_list, target_schema_owner_list, max_workers, log_table_name)

['_eo_amperity:dr_eo_amperity', 'accuweather_daily_and_hourly_forecasts_u_s_postal_codes_sample:dr_accuweather', 'test_not:dr_test_not', 'satya_share:satya_dr_catalog'] ['dr_eo_amperity.bronze:grp_test_clt_usa', 'dr_eo_amperity.silver:grp_test_clt_usa', 'dr_eo_amperity.default:grp_test_clt_usa'] 5 users.satyendranath_sure.dr_log_table_name


In [0]:
# for catalog_pair in catalog_pair_names_list:
#     second_value = catalog_pair.split(":")[1]
#     print(second_value)
#     spark.sql(f"DROP CATALOG IF EXISTS `{second_value}` CASCADE")
# spark.sql(f"DROP table if exists `{log_table_name}`")

dr_eo_amperity
dr_accuweather
dr_test_not
satya_dr_catalog


DataFrame[]

In [0]:
import traceback
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, ArrayType
from datetime import datetime

# Get the current notebook name
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
notebook_name = notebook_path.split('/')[-1]

# Schema definition for log table
log_table_schema = StructType([
    StructField("notebook_name", StringType(), True),
    StructField("entity_type", StringType(), True),
    StructField("entity_name", StringType(), True),
    StructField("action", StringType(), True),
    StructField("status", StringType(), True),
    StructField("message", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("results_data", ArrayType(StructType([
        StructField("key", StringType(), True),
        StructField("value", StringType(), True)
    ])), True)
])


# Create log table
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {log_table_name}
    (
        notebook_name STRING,
        entity_type STRING,
        entity_name STRING,
        action STRING,
        status STRING,
        message STRING,
        timestamp TIMESTAMP,
        results_data ARRAY<STRUCT<key: STRING, value: STRING>>
    )
""")

def insert_log_entry(log_data):
    try:
        # Add the notebook_name to the log_data dictionary
        log_data["notebook_name"] = notebook_name
        spark.createDataFrame([log_data], schema=log_table_schema).write.mode("append").saveAsTable(log_table_name)
    except Exception as e:
        print(f"Failed to insert log entry: {log_data}. Error: {e}")

def execute_and_log_sql(sql_command, entity_type, entity_name, action, success_message, error_message):
    try:
        results_df = spark.sql(sql_command)
        results = [
            {"key": col_name, "value": str(row[col_name])}
            for row in results_df.collect()
            for col_name in row.__fields__
        ]
        log_entry = {
            "entity_type": entity_type,
            "entity_name": entity_name,
            "action": action,
            "status": "success",
            "message": success_message,
            "timestamp": datetime.now(),
            "results_data": results
        }
    except Exception as e:
        log_entry = {
            "entity_type": entity_type,
            "entity_name": entity_name,
            "action": action,
            "status": "error",
            "message": f"{error_message}. Error: {e}",
            "timestamp": datetime.now(),
            "results_data": []
        }
    
    insert_log_entry(log_entry)


def create_catalog_if_not_exists(target_catalog):
    sql_command = f"CREATE CATALOG IF NOT EXISTS `{target_catalog}`"
    execute_and_log_sql(
        sql_command, 
        "catalog", 
        f"`{target_catalog}`", 
        "create",
        f"Catalog `{target_catalog}` created or already exists.",
        f"Failed to create catalog `{target_catalog}`."
    )

def get_source_schemas(source_catalog):
    try:
        return [row.databaseName for row in spark.sql(f"SHOW DATABASES IN `{source_catalog}`").collect()], None
    except Exception as e:
        return None, f"Failed to get schemas from source catalog `{source_catalog}`. Error: {e}"

def create_schema_and_set_owner(target_catalog, schema, target_schema_owner_list):
    target_schema_fqn = f"`{target_catalog}`.`{schema}`"
    
    # Create schema
    execute_and_log_sql(
        f"CREATE SCHEMA IF NOT EXISTS {target_schema_fqn}",
        "schema",
        target_schema_fqn,
        "create",
        f"Schema {target_schema_fqn} created or already exists.",
        f"Failed to create schema {target_schema_fqn}."
    )

    # Set ownership
    owner_info = f"{target_catalog}.{schema}"
    for owner_mapping in target_schema_owner_list:
        schema_to_match, owner_group = owner_mapping.split(':')
        if owner_info == schema_to_match:
            execute_and_log_sql(
                f"ALTER SCHEMA {target_schema_fqn} OWNER TO `{owner_group}`",
                "schema",
                target_schema_fqn,
                "change_ownership",
                f"Ownership of schema {target_schema_fqn} changed to `{owner_group}`.",
                f"Failed to change ownership of schema {target_schema_fqn}."
            )
            return
    insert_log_entry({
        "entity_type": "schema",
        "entity_name": target_schema_fqn,
        "action": "change_ownership",
        "status": "skipped",
        "message": "No ownership mapping found for this schema. Skipping ownership change.",
        "timestamp": datetime.now(),
        "results_data": []
    })

def get_source_tables_and_views(source_catalog, schema_name):
    try:
        return spark.sql(f"SHOW TABLES IN `{source_catalog}`.`{schema_name}`").collect(), None
    except Exception as e:
        return None, f"Failed to get tables from source schema `{source_catalog}`.`{schema_name}`. Error: {e}"

def process_table_or_view(source_catalog, target_catalog, schema_name, table_row):
    table_name = table_row.tableName
    is_temporary = table_row.isTemporary
    source_table_fqn = f"`{source_catalog}`.`{schema_name}`.`{table_name}`"
    target_table_fqn = f"`{target_catalog}`.`{schema_name}`.`{table_name}`"
    
    print(f"Processing table/view: {source_table_fqn}")

    if is_temporary:
        insert_log_entry({
            "entity_type": "table/view",
            "entity_name": source_table_fqn,
            "action": "clone/create",
            "status": "skipped",
            "message": f"Skipping temporary table/view {source_table_fqn}.",
            "timestamp": datetime.now(),
            "results_data": []
        })
        return

    try:
        desc_info = {
            row.col_name: row.data_type 
            for row in spark.sql(f"DESCRIBE TABLE EXTENDED {source_table_fqn}").collect()
            if row.col_name and row.data_type
        }
        
        if desc_info.get("Type") == "VIEW":
            view_text = desc_info.get("View Text")
            if view_text:
                execute_and_log_sql(
                    f"CREATE OR REPLACE VIEW {target_table_fqn} AS {view_text}",
                    "view",
                    target_table_fqn,
                    "create",
                    f"View {target_table_fqn} created successfully.",
                    f"Failed to create view {target_table_fqn}."
                )
            else:
                insert_log_entry({
                    "entity_type": "view",
                    "entity_name": source_table_fqn,
                    "action": "create",
                    "status": "error",
                    "message": "View text definition not found.",
                    "timestamp": datetime.now(),
                    "results_data": []
                })
        else:
            execute_and_log_sql(
                f"CREATE OR REPLACE TABLE {target_table_fqn} CLONE {source_table_fqn}",
                "table",
                target_table_fqn,
                "clone",
                f"Table {target_table_fqn} cloned successfully.",
                f"Failed to clone table {target_table_fqn}."
            )
    except Exception as e:
        insert_log_entry({
            "entity_type": "table/view",
            "entity_name": source_table_fqn,
            "action": "clone/create",
            "status": "error",
            "message": f"Failed to process table/view {source_table_fqn}. Error: {e}",
            "timestamp": datetime.now(),
            "results_data": []
        })

def collect_all_table_tasks(catalog_pair_names_list, target_schema_owner_list):
    all_tasks = []
    
    for catalog_pair_string in catalog_pair_names_list:
        try:
            source_catalog, target_catalog = catalog_pair_string.split(':')
            print(f"Processing catalog pair: {source_catalog} -> {target_catalog}")
            insert_log_entry({
                "entity_type": "catalog_pair",
                "entity_name": f"{source_catalog} -> {target_catalog}",
                "action": "process",
                "status": "start",
                "message": f"Starting processing for catalog pair: {source_catalog} -> {target_catalog}",
                "timestamp": datetime.now(),
                "results_data": []
            })

            create_catalog_if_not_exists(target_catalog)
            schemas, error = get_source_schemas(source_catalog)
            
            if error:
                insert_log_entry({
                    "entity_type": "catalog",
                    "entity_name": f"`{source_catalog}`",
                    "action": "get_schemas",
                    "status": "error",
                    "message": error,
                    "timestamp": datetime.now(),
                    "results_data": []
                })
                continue

            for schema in schemas:
                if schema == 'information_schema':
                    print(f"  Skipping schema: `{source_catalog}`.`{schema}`")
                    insert_log_entry({
                        "entity_type": "schema",
                        "entity_name": f"`{source_catalog}`.`{schema}`",
                        "action": "skip",
                        "status": "success",
                        "message": "Skipping `information_schema`.",
                        "timestamp": datetime.now(),
                        "results_data": []
                    })
                    continue

                print(f"  Processing schema: `{source_catalog}`.`{schema}` -> `{target_catalog}`.`{schema}`")
                create_schema_and_set_owner(target_catalog, schema, target_schema_owner_list)
                tables, tables_error = get_source_tables_and_views(source_catalog, schema)
                
                if tables_error:
                    insert_log_entry({
                        "entity_type": "schema",
                        "entity_name": f"`{source_catalog}`.`{schema}`",
                        "action": "get_tables",
                        "status": "error",
                        "message": tables_error,
                        "timestamp": datetime.now(),
                        "results_data": []
                    })
                    continue

                for table_row in tables:
                    all_tasks.append((source_catalog, target_catalog, schema, table_row))

            insert_log_entry({
                "entity_type": "catalog_pair",
                "entity_name": f"{source_catalog} -> {target_catalog}",
                "action": "process",
                "status": "sequential_complete",
                "message": f"Finished sequential processing for catalog pair: {source_catalog} -> {target_catalog}",
                "timestamp": datetime.now(),
                "results_data": []
            })

        except Exception as e:
            insert_log_entry({
                "entity_type": "catalog_pair",
                "entity_name": f"Processing of {catalog_pair_string}",
                "action": "unhandled_error",
                "status": "error",
                "message": f"An unhandled error occurred: {e}\n{traceback.format_exc()}",
                "timestamp": datetime.now(),
                "results_data": []
            })

    return all_tasks

def main(catalog_pair_names_list, target_schema_owner_list, max_workers):
    print("Starting sequential catalog and schema processing...")
    
    # Collect all table tasks sequentially
    all_tasks = collect_all_table_tasks(catalog_pair_names_list, target_schema_owner_list)
    print(f"Collected {len(all_tasks)} tables/views to process.")
    # print(all_tasks)

    # Process all tables in parallel
    print("Starting parallel table processing...")
    # with ThreadPoolExecutor(max_workers=max_workers) as executor:
    #     executor.map(
    #         lambda task: process_table_or_view(*task),
    #         all_tasks
    #     )
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks at once and store futures
        futures = [executor.submit(process_table_or_view, *task) for task in all_tasks]
        # Wait for all tasks to complete
        concurrent.futures.wait(futures)
        # Optionally, check for exceptions
        for future in futures:
            try:
                future.result()  # Ensure any exceptions are raised
            except Exception as e:
                print(f"Task generated an exception: {e}")

    print("Processing complete.")

# Run job
main(catalog_pair_names_list, target_schema_owner_list, max_workers)

Starting sequential catalog and schema processing...
Processing catalog pair: _eo_amperity -> dr_eo_amperity
  Processing schema: `_eo_amperity`.`bronze` -> `dr_eo_amperity`.`bronze`
  Processing schema: `_eo_amperity`.`default` -> `dr_eo_amperity`.`default`
  Skipping schema: `_eo_amperity`.`information_schema`
  Processing schema: `_eo_amperity`.`silver` -> `dr_eo_amperity`.`silver`
Processing catalog pair: accuweather_daily_and_hourly_forecasts_u_s_postal_codes_sample -> dr_accuweather
  Processing schema: `accuweather_daily_and_hourly_forecasts_u_s_postal_codes_sample`.`forecast` -> `dr_accuweather`.`forecast`
  Skipping schema: `accuweather_daily_and_hourly_forecasts_u_s_postal_codes_sample`.`information_schema`
Processing catalog pair: test_not -> dr_test_not
Processing catalog pair: satya_share -> satya_dr_catalog
  Skipping schema: `satya_share`.`information_schema`
  Processing schema: `satya_share`.`satyendranath_sure` -> `satya_dr_catalog`.`satyendranath_sure`
Collected 35 t

In [0]:
# Display logs for the last 24 hours
print(f"\n--- Displaying logs from {log_table_name} for the last 24 hours ---")
# past_24_hours = datetime.now() - timedelta(hours=24)
log_query_df = spark.sql(f"""
    SELECT *
    FROM {log_table_name}
    ORDER BY timestamp DESC
""")

display(log_query_df)

In [0]:
# %sql
# drop table 
# -- select * from 
# -- users.satyendranath_sure.dr_log_table_name